In [1]:
import pandas as pd
import numpy as np


In [46]:
#D:\Szakdolgozat_data\nba-movement-data-master\data\csv_fix
tracking = pd.read_csv(r'D:\Szakdolgozat_data\nba-movement-data-master\data\csv_fix\0021500001.csv')
events = pd.read_csv(r'D:\Szakdolgozat_data\Basketball_reference_data\shot_and_stat.csv')
#if 1by1  pd.read_csv(r'D:\Szakdolgozat_data\nba-movement-data-master\data\events\0021500001.csv')

In [47]:
events =events.drop(columns = ['MINUTES_REMAINING','SECONDS_REMAINING','Unnamed: 0.1', 'Name', 'Namecheck', 'Namecheck2', 'Namecheck3'])
events.head(5)


,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,PERIOD,ACTION_TYPE,SHOT_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,...,True Shooting,AST%,OWS,DWS,WS/48,OBPM,DBPM,Height,Weight,SECLEFT
0,21500014,20,202083,Wesley Matthews,1610612742,1,Jump Shot,3PT Field Goal,Above the Break 3,Right Side Center(RC),...,0.586,10.8,4.0,2.2,0.147,2.0,1.1,195.58,99.79024,544
1,21500014,168,202083,Wesley Matthews,1610612742,2,Jump Shot,3PT Field Goal,Above the Break 3,Left Side Center(LC),...,0.586,10.8,4.0,2.2,0.147,2.0,1.1,195.58,99.79024,662
2,21500014,217,202083,Wesley Matthews,1610612742,2,Jump Shot,3PT Field Goal,Above the Break 3,Right Side Center(RC),...,0.586,10.8,4.0,2.2,0.147,2.0,1.1,195.58,99.79024,490
3,21500014,340,202083,Wesley Matthews,1610612742,3,Jump Shot,3PT Field Goal,Above the Break 3,Right Side Center(RC),...,0.586,10.8,4.0,2.2,0.147,2.0,1.1,195.58,99.79024,657
4,21500014,358,202083,Wesley Matthews,1610612742,3,Jump Shot,2PT Field Goal,Mid-Range,Right Side(R),...,0.586,10.8,4.0,2.2,0.147,2.0,1.1,195.58,99.79024,573


In [48]:
events.columns

Index(['GAME_ID', 'GAME_EVENT_ID', 'PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID',
       'PERIOD', 'ACTION_TYPE', 'SHOT_TYPE', 'SHOT_ZONE_BASIC',
       'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_DISTANCE', 'LOC_X', 'LOC_Y',
       'SHOT_MADE_FLAG', 'Unnamed: 0', 'Position', 'Minutes Played', 'FG%',
       'Avg dist', '% of shots 2P', '% of shots 0-3', '% of shots 3-10',
       '% of shots 10-16', '% of shots 16-3P', '% of shots 3P', 'FG% from 2P',
       'FG% from 0-3', 'FG% from 3-10', 'FG% from 10-16', 'FG% from 16-3P',
       'FG% from 3P', '% assisted 2P', '% assisted 3P', '% of 3P corner',
       'corner 3P %', '2P%', '3P%', 'FT%', 'True Shooting', 'AST%', 'OWS',
       'DWS', 'WS/48', 'OBPM', 'DBPM', 'Height', 'Weight', 'SECLEFT'],
      dtype='object')

In [56]:
events1game = events.loc[events['GAME_ID'] == 21500001]

In [2]:
def p_num_from_tr_id(tracking_record, p_tr_id):
    for num in range(1,11):
        if tracking_record[f'p{num}_player_id'] == p_tr_id:
            return num

def transform_coord_pair(x,y):
    if x > 47 :
        y = 50 - y
        x = 94 - x
    y = 250 * (1 - (y - 0)/(50 - 0)) + -250 * ((y - 0)/(50 - 0))
    x = -47.5 * (1 - (x - 0)/(47 - 0)) + 422.5 * ((x - 0)/(47 - 0))

    return x,y

def calc_dist(x_1, y_1, x_2, y_2):
    return np.sqrt((x_2 - x_1) ** 2 + (y_2 - y_1) ** 2)

def get_closest_frame(x,y,shooter,tracking):
    min_dist = 999
    min_frame_i = np.nan
    for idx, frame in tracking.iterrows():
        #transform coords
        player_num = p_num_from_tr_id(frame,shooter)
        if player_num:
            px,py = transform_coord_pair(frame[f'p{player_num}_x'],frame[f'p{player_num}_y'])
        else:
            px,py = transform_coord_pair(frame['b_x'],frame['b_y'])
        bx,by = transform_coord_pair(frame['b_x'],frame['b_y'])
        #calc distance
        pdist = calc_dist(px,py, x,y)
        bdist = calc_dist(bx,by, x,y)
        
        dist = pdist*0.6 + bdist*0.4
        #update if smaller than min
        if dist < min_dist:
            min_dist = dist
            min_frame_i = idx
    return min_frame_i#min_dist, min_frame_i

def get_frame_ids(tracking,events):
    frame_ids = []
    for idx, event in events.iterrows():
        timeleft = event['SECLEFT'] #event['MINUTES_REMAINING']*60 + event['SECONDS_REMAINING']
        shooter = event['PLAYER_ID']
        q = event['PERIOD']
        x,y = event['LOC_Y'], event['LOC_X']
        tr_window = tracking[(tracking['quarter'] == q) & (tracking['game_clock'] <= timeleft+4) & (tracking['game_clock']>=timeleft-1)]
        min_i = get_closest_frame(x,y,shooter,tr_window)
        frame_ids.append(min_i) 
    return frame_ids

def sync(tracking,events):
    frame_ids = get_frame_ids(tracking,events)
    try:
        events['frame_id'] = frame_ids
    except ValueError:
        print('Value error while syncing')
        return pd.DataFrame()
    tracking['frame_id'] = tracking.index
    
    merged = pd.merge(events, tracking, how='left', on=['frame_id'], suffixes=('_event', '_tracking'))
    
    return merged

def dist_to_defenders(df):
    avg_dists=[]
    min_dists=[]
    for idx,val in df.iterrows():
        shooter_num = p_num_from_tr_id(val,val['PLAYER_ID'])
        if shooter_num:
            sh_x,sh_y = val[f'p{shooter_num}_x'],val[f'p{shooter_num}_y']
        else:
            sh_x,sh_y = val['b_x'],val['b_y']
        shooter_team = val['TEAM_ID']
        dists = []
        for num in range(1,11):
            if val[f'p{num}_team_id'] != shooter_team:
                dist = calc_dist(sh_x,sh_y,val[f'p{num}_x'],val[f'p{num}_y'])
                dists.append(dist)
        avg_dist = sum(dists)/len(dists)
        min_dist = min(dists)
        avg_dists.append(avg_dist)
        min_dists.append(min_dist)
    df['AVG_DIST_TO_SHOOTER'] = avg_dists
    df['CLOSEST_DEFENDER_DIST'] = min_dists
    return df

In [61]:
merged = sync(tracking,events1game)

C:\Users\Acer\AppData\Local\Temp\ipykernel_1212\992274319.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events['frame_id'] = frame_ids


In [62]:
df1game = dist_to_defenders(merged)

In [67]:
df1game.head(5)

,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,PERIOD,ACTION_TYPE,SHOT_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,...,p9_x,p9_y,p9_z,p10_team_id,p10_player_id,p10_x,p10_y,p10_z,AVG_DIST_TO_SHOOTER,CLOSEST_DEFENDER_DIST
0,21500001,138,203145,Kent Bazemore,1610612737,2,Cutting Layup Shot,2PT Field Goal,Restricted Area,Center(C),...,70.71885,40.87657,0.0,1.610613e+09,1626169.0,86.27102,24.20541,0.0,13.179578,3.813211
1,21500001,152,203145,Kent Bazemore,1610612737,2,Driving Finger Roll Layup Shot,2PT Field Goal,Restricted Area,Center(C),...,88.70172,36.16600,0.0,1.610613e+09,1626169.0,83.81907,28.10703,0.0,6.558750,2.395280
2,21500001,260,203145,Kent Bazemore,1610612737,3,Jump Shot,3PT Field Goal,Above the Break 3,Left Side Center(LC),...,16.20993,33.46774,0.0,1.610613e+09,203083.0,16.88875,20.88557,0.0,15.880111,10.852580
3,21500001,81,203471,Dennis Schroder,1610612737,1,Cutting Layup Shot,2PT Field Goal,Restricted Area,Center(C),...,78.15901,24.20540,0.0,1.610613e+09,1626169.0,88.33127,20.12118,0.0,10.012759,4.986705
4,21500001,92,203471,Dennis Schroder,1610612737,1,Driving Layup Shot,2PT Field Goal,Restricted Area,Center(C),...,87.78286,25.58873,0.0,1.610613e+09,1626169.0,72.92065,21.72851,0.0,9.217019,3.344172


In [69]:
print(df1game.columns.tolist())


['GAME_ID', 'GAME_EVENT_ID', 'PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'PERIOD', 'ACTION_TYPE', 'SHOT_TYPE', 'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_DISTANCE', 'LOC_X', 'LOC_Y', 'SHOT_MADE_FLAG', 'Unnamed: 0', 'Position', 'Minutes Played', 'FG%', 'Avg dist', '% of shots 2P', '% of shots 0-3', '% of shots 3-10', '% of shots 10-16', '% of shots 16-3P', '% of shots 3P', 'FG% from 2P', 'FG% from 0-3', 'FG% from 3-10', 'FG% from 10-16', 'FG% from 16-3P', 'FG% from 3P', '% assisted 2P', '% assisted 3P', '% of 3P corner', 'corner 3P %', '2P%', '3P%', 'FT%', 'True Shooting', 'AST%', 'OWS', 'DWS', 'WS/48', 'OBPM', 'DBPM', 'Height', 'Weight', 'SECLEFT', 'frame_id', 'game_id', 'event_id', 'game_clock', 'shot_clock', 'quarter', 'b_team_id', 'b_player_id', 'b_x', 'b_y', 'b_z', 'p1_team_id', 'p1_player_id', 'p1_x', 'p1_y', 'p1_z', 'p2_team_id', 'p2_player_id', 'p2_x', 'p2_y', 'p2_z', 'p3_team_id', 'p3_player_id', 'p3_x', 'p3_y', 'p3_z', 'p4_team_id', 'p4_player_id', 'p4_x', 'p4_y'

In [13]:
#For all games
dfs = {}
events = pd.read_csv(r'D:\Szakdolgozat_data\Basketball_reference_data\shot_and_stat.csv')
events =events.drop(columns = ['MINUTES_REMAINING','SECONDS_REMAINING','Unnamed: 0.1', 'Name', 'Namecheck', 'Namecheck2', 'Namecheck3'])
for game in range(21500001, 21500152):
    try:
        tracking = pd.read_csv(r'D:\Szakdolgozat_data\nba-movement-data-master\data\csv_fix\00{0}.csv'.format(game))
        events1game = events.loc[events['GAME_ID'] == game]
        merged = sync(tracking,events1game)
        dfs[game] = dist_to_defenders(merged)
    except OSError as e:
        continue
    
    

C:\Users\Acer\AppData\Local\Temp\ipykernel_12372\992274319.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events['frame_id'] = frame_ids
C:\Users\Acer\AppData\Local\Temp\ipykernel_12372\992274319.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events['frame_id'] = frame_ids
C:\Users\Acer\AppData\Local\Temp\ipykernel_12372\992274319.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

C:\Users\Acer\AppData\Local\Temp\ipykernel_12372\992274319.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events['frame_id'] = frame_ids
C:\Users\Acer\AppData\Local\Temp\ipykernel_12372\992274319.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events['frame_id'] = frame_ids
C:\Users\Acer\AppData\Local\Temp\ipykernel_12372\992274319.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

C:\Users\Acer\AppData\Local\Temp\ipykernel_12372\992274319.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events['frame_id'] = frame_ids
C:\Users\Acer\AppData\Local\Temp\ipykernel_12372\992274319.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events['frame_id'] = frame_ids
C:\Users\Acer\AppData\Local\Temp\ipykernel_12372\992274319.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

C:\Users\Acer\AppData\Local\Temp\ipykernel_12372\992274319.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events['frame_id'] = frame_ids
C:\Users\Acer\AppData\Local\Temp\ipykernel_12372\992274319.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events['frame_id'] = frame_ids
C:\Users\Acer\AppData\Local\Temp\ipykernel_12372\992274319.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

C:\Users\Acer\AppData\Local\Temp\ipykernel_12372\992274319.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events['frame_id'] = frame_ids
C:\Users\Acer\AppData\Local\Temp\ipykernel_12372\992274319.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events['frame_id'] = frame_ids
C:\Users\Acer\AppData\Local\Temp\ipykernel_12372\992274319.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

C:\Users\Acer\AppData\Local\Temp\ipykernel_12372\992274319.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events['frame_id'] = frame_ids
C:\Users\Acer\AppData\Local\Temp\ipykernel_12372\992274319.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events['frame_id'] = frame_ids
C:\Users\Acer\AppData\Local\Temp\ipykernel_12372\992274319.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

C:\Users\Acer\AppData\Local\Temp\ipykernel_12372\992274319.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events['frame_id'] = frame_ids
C:\Users\Acer\AppData\Local\Temp\ipykernel_12372\992274319.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events['frame_id'] = frame_ids
C:\Users\Acer\AppData\Local\Temp\ipykernel_12372\992274319.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

In [16]:
game1to150 = pd.concat(dfs)

In [21]:
game1to150.to_csv(r'D:\Szakdolgozat_data\Features_jupyter\features150ig.csv')